In [ ]:
import requests
import requests.cookies
import json
import os
import ssl
import certifi
import websockets
import re


class TradeRepublicApi:
    HOST = "https://api.traderepublic.com"
    WSS_HOST = "wss://api.traderepublic.com"
    HEADERS = {
        "User-Agent": "Mozilla/5.0 (Linux; Android 10; SM-G960F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.181 Mobile Safari/537.36",
    }

    def __init__(self, phone: str, pin: str):
        self.phone = phone
        self.pin = pin
        self.session = requests.Session()
        self.session.headers.update(self.HEADERS)

    def login(self):
        response = self.session.post(
            url=f"{self.HOST}/api/v1/auth/web/login",
            json={"phoneNumber": self.phone, "pin": self.pin},
        )

        assert response.status_code == 200, response.text

        r = response.json()

        assert "processId" in r, response.text

        self.process_id = r["processId"]
        print(f"Login process ID: {self.process_id}")
        

    def enter_2fa_code(self, code: str):
        url = f"{self.HOST}/api/v1/auth/web/login/{self.process_id}/{code}"
        response = self.session.post(
            url=url,
        )

        assert response.status_code == 200, response.text

        print(response)
        print(response.text)
    
    def refresh_session(self):
        response = self.session.get(
            url=f"{self.HOST}/api/v1/auth/session",
        )

        assert response.status_code == 200, response.text

        print(response)
        print(response.text)

        self._save_session()

    def is_logged_in(self):
        response = self.session.get(url=f"{self.HOST}/api/v2/auth/account")

        return response.status_code == 200
    

    def account(self):
        response = self.session.get(url=f"{self.HOST}/api/v2/auth/account")

        assert response.status_code == 200, response.text

        print(response.text)
        
    async def connect_websocket(self):
        ssl_context = ssl.create_default_context(cafile=certifi.where())
        additional_headers = {
            "Cookie": "; ".join(
                [
                    f"{cookie.name}={cookie.value}"
                    for cookie in self.session.cookies
                    if cookie.domain.endswith("traderepublic.com")
                ]
            ),
        }

        self.ws = await websockets.connect(
            uri=f"{self.WSS_HOST}",
            additional_headers=additional_headers,
            ssl=ssl_context,
        )
        await self.ws.send(
            f"connect 31 {json.dumps({
                "locale": "en",
                "platformId": "webtrading",
                "platformVersion": "chrome - 131.0.0",
                "clientId": "app.traderepublic.com",
                "clientVersion": "3.130.0",
            })}"
        )
        response = await self.ws.recv()        
        assert "connected" == response, response     
        
    async def timeline_transactions(self):
        await self.ws.send(
            f"sub 165 {json.dumps({'type': 'timelineTransactions', "token": self.process_id})}"
        )
        response = await self.ws.recv()
        
        json_string = re.search(r"\d+ \w+ (.*)", response).group(1)
        transactions = json.loads(json_string)
        
        assert "errors" not in transactions, transactions
        
        return json.loads(json_string)


with open("creds", "r", encoding="utf-8") as f:
    phone = f.readline().strip()
    pin = f.readline().strip()

api = TradeRepublicApi(phone, pin)

api.login()
code = input("Enter 2FA code: ")
api.enter_2fa_code(code)

await api.connect_websocket()
transactions = await api.timeline_transactions()

# write to file
with open("transactions.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(transactions, indent=4))

In [ ]:
import json
import pandas as pd

# load from file
with open("transactions.json", "r", encoding="utf-8") as f:
    transactions = json.load(f)
print(transactions["cursors"])
print(transactions["startingTransactionId"])

df = pd.DataFrame(transactions["items"])
df

# write to file
df.to_csv("transactions.csv", index=False)